# Evaluating GPT 3.5 on sentiment analysis
In this notebook, the GPT 3.5 model from OpenAI will be evaluated on a sentiment analysis task. The model will be applied through an API, and will be asked to categorize ratings from different exchange students into the sentiments "positive", "neutral" or "negative". Evaluation will be conducted using a confusion matrix, which will compare the sentiments predicted by GPT with manually labeled sentiments. Additionally, precision, recall, and F1 scores will be computed to provide a more comprehensive assessment of the model's performance.

# Data import 
In this section every labelled data from the student data set is imported. As GPT will be evaluated on its sentiment categorization, only labelled datasets are utilized.

In [2]:
import pandas as pd

# import of data
#student_data_unpr = pd.read_excel("../data/subset_studium_german.xlsx")
#student_data_unpr = student_data_unpr.drop("Unnamed: 0", axis=1)
#student_data_unpr = student_data_unpr.drop("Studium_Comment", axis=1)

#Reduction to labeled entries
#student_data_unpr = student_data_unpr.drop(student_data_unpr.index[1020:], axis=0)

#Creation of a copy and reduction to column "Studium"
student_data = student_data_unpr
student_data = student_data.drop("Studium_Label", axis=1)



KeyError: "['Studium_Label'] not found in axis"

# [1] OpenAI GPT3.5 application

Firstly, GPT is presented with a prompt tasking it to classify incoming inputs into one of three sentiment categories: "positive", "neutral", or "negative". This process occurs iteratively, with the prompt being included with each new request sent to the model. To enhance the objectivity and consistency of results, parameters such as temperature and seed are configured.

The temperature parameter serves to temper the creativity of GPT's responses, promoting more objective outcomes. Meanwhile, the seed parameter is adjustable to any chosen value, ensuring greater consistency and determinism across multiple API calls, provided that the seed value remains constant.

In [ ]:
#Opening the stream to OpenAI
from openai import OpenAI
client = OpenAI()

In [ ]:
#Prompt for GPT
initial_message = {"role": "user", "content": "You are my sentiment assistant. I want you to analyze my text and categorize it into the three sentiments: positive, neutral, negative. Only respond with either positive, neutral or negative."}

results=[]
results_df = []
count = 1

# Sending labeling requests to GPT
for index, eintrag in student_data.iterrows():
    for inhalt in eintrag:
        count=+1
        display(count)
        response = client.chat.completions.create(
        model="gpt-3.5-turbo",
            messages=[
                initial_message,
                {"role": "user", "content": inhalt}
            ],
            #Setting temperature to ensure objective responses
            temperature=0,
            #Setting the seed parameter to ensure more consistent results
            seed = 123
        )
        
        # Saving of responses 
        results.append({"index": index, "sentimentPrediction": response.choices[0].message.content.lower()})

# Conversion of responses into dataframe
results_df = pd.DataFrame(results)




# [1.1] First evaluation
During the evaluation, a confusion matrix is employed to compare the predicted labels against the actual labels. It becomes apparent that GPT exhibits a bias towards categorizing inputs into more positive sentiments, evidenced by a precision of 1.0 for positive labels. Furthermore, the model achieves only an overall F1 score of 0.57. Notably, only 10 out of 132 negative datasets are correctly labeled, indicating a significant challenge in accurately identifying negative sentiments.

In [ ]:
#Merge target labels with predicted labels and dropping of duplicated row
results_df = results_df.drop("index", axis=1)
student_data = pd.concat([student_data_unpr, results_df], axis=1)
student_data = student_data[student_data["Studium_Label"].notnull()]


#Convertion of all values into strings
for x in student_data:
    student_data[x] = student_data[x].astype(str)



In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, classification_report
import matplotlib.pyplot as plt

#Creation of confusion matrix
confusion_mat = confusion_matrix(student_data['Studium_Label'], student_data['sentimentPrediction'], labels=['positive', 'negative', 'neutral'])
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_mat, display_labels=['positive', 'negative', 'neutral'])
disp.plot()
plt.show()

#Accuracy
print(f1_score(student_data['Studium_Label'], student_data['sentimentPrediction'], labels=['positive', 'negative', 'neutral'], average='micro'))

#Creation of presicion, recall, F1 and support
print(classification_report(
    student_data['Studium_Label'],
    student_data['sentimentPrediction'],
    output_dict=False,
    target_names=['positive', 'negative', 'neutral']
))

# [2] Alteration of prompt towards more negative categorization
As demonstrated in the evaluation, GPT 3.5 exhibited a bias towards categorizing inputs into more positive sentiments. To improve the accuracy and achieve a more negative categorization tendency, GPT was presented with an altered prompt designed to influence its categorization towards negativity.

The same parameters, with temperature set to 0 and seed set to 123 as in the initial approach, were utilized for consistency.

In [ ]:
#Prompt for GPT
initial_message = {"role": "user", "content": "You are my sentiment assistant. I want you to analyze my text and categorize it into the three sentiments: positive, neutral, negative. However, if a text has negative aspects, have a higher tendency towards a negative rating. Only respond with either positive, neutral or negative."}

#Creation of a new data set
student_data_altered = student_data_unpr
student_data_altered = student_data_unpr.drop("Studium_Label", axis=1)

results=[]
results_df = []

# Sending labeling requests to GPT
for index, eintrag in student_data_altered.iterrows():
    for inhalt in eintrag:
        response = client.chat.completions.create(
        model="gpt-3.5-turbo",
            messages=[
                initial_message,
                {"role": "user", "content": inhalt}
            ],
            #Setting temperature to ensure objective responses
            temperature=0,
            #Setting the seed parameter to ensure more consistent results
            seed = 123
        )
        
        # Saving of responses 
        results.append({"index": index, "alteredSentimentPrediction": response.choices[0].message.content.lower()})

# Conversion of responses into dataframe
results_df = pd.DataFrame(results)



# [2.1] Second evaluation 
In the second evaluation, GPT persists in demonstrating a bias towards positive categorization, with a reduced precision of 0.78 for positive sentiments, compared to the first evaluation. However, despite this bias, the overall F1 score shows an improvement, increasing by 4%. Nevertheless, GPT continues to struggle with accurately categorizing negative sentiments, correctly identifying only 27 out of 132 negatively labeled datasets.



In [ ]:
#Merging target labels with predicted labels and dropping of duplicated row
results_df = results_df.drop("index", axis=1)
student_data_altered = pd.concat([student_data_unpr, results_df], axis=1)
student_data_altered = student_data_altered[student_data_altered["Studium_Label"].notnull()]

#Adding of altered predictions to a dataframe in order to save all predictions in an excel
altered_prediction = student_data_altered["alteredSentimentPrediction"]
student_data = pd.concat([student_data, altered_prediction], axis=1)


#Convertion of all values into strings
for x in student_data_altered:
    student_data[x] = student_data[x].astype(str)



In [ ]:
#saving of data into an excel sheet
#student_data.to_excel("Evaluation.xlsx", index=False) 

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, classification_report
import matplotlib.pyplot as plt

#Creation of confusion matrix
confusion_mat = confusion_matrix(student_data_altered['Studium_Label'], student_data_altered['alteredSentimentPrediction'], labels=['positive', 'negative', 'neutral'])
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_mat, display_labels=['positive', 'negative', 'neutral'])
disp.plot()
plt.show()

#Accuracy
print(f1_score(student_data_altered['Studium_Label'], student_data_altered['alteredSentimentPrediction'], labels=['positive', 'negative', 'neutral'], average='micro'))

#Creation of presicion, recall, F1 and support
print(classification_report(
    student_data_altered['Studium_Label'],
    student_data_altered['alteredSentimentPrediction'],
    output_dict=False,
    target_names=['positive', 'negative', 'neutral']
))

# [3] One shot learning with altered prompt
As demonstrated in the evaluation, GPT 3.5 exhibited a bias towards categorizing inputs into more positive sentiments. To improve the accuracy and achieve a more negative categorization tendency, GPT was presented with an altered prompt designed to influence its categorization towards negativity.

The same parameters, with temperature set to 0 and seed set to 123 as in the initial approach, were utilized for consistency

In [ ]:
#Prompt for GPT
initial_message = {"role": "user", "content": "You are my sentiment assistant. I want you to analyze my text and categorize it into the three sentiments: positive, neutral, negative. However, if a text has negative aspects, have a higher tendency towards a negative rating. This is an example for a negative labeled dataset: 'Die City University of Hong Kong hat eine große Kursauswahl. Die Kurse gehen für gewöhnlich drei Stunden lang und sie haben einen hohen Workload (vor allem im letzten Drittel der Vorlesungszeit). Das Kursniveau generell nicht auf dem von Mannheim. Es könnte ein kultureller Unterschied sein, aber die Fragestellungen für Essays und Gruppenarbeiten sind oftmals sehr vage und breit gefasst. Auch die Bewertung ist intransparent, man erhält kein Feedback bevor man die Endnote für den Kurs erhält. Mein Semester ging von Ende August bis Ende November (Vorlesungszeit). Nach einer Woche Pause beginnt die zwei wöchige Prüfungszeit und endet vor Weihnachten. Ich hatte jedoch keine Klausur in der Prüfungsphase, alle Abgaben (Essays, Gruppenberichte) und Präsentationen/Tests liefen unter dem Semester.Die Mensen der CityU sind sehr empfehlenswert. Westliches und asiatisches Essen ist beides vorhanden. Vor allem die Desserts in AC1 sind eine große Empfehlung.Das Sprachniveau war sehr durchmischt, sowohl bei Professoren, als auch bei Studenten. Teils waren Fragestellungen in Prüfungen sprachlich schwer verständlich. In Restaurants kommt man gut damit aus auf die Bilder/Karte zu zeigen.' This is an example for a neutral dataset: 'Da ich während Corona vor Ort war habe ich die Uni nur zweimal besucht. Die restlichen Monate hatte ich nur Online Vorlesungen. Von den Professoren und den Mitarbeitern kann ich sagen, dasss alle sehr nett und hilfsbereit waren. Falls ihr Probleme oder Fragen habt sendet einfach eine Email an das Studienbüro und die werden euch helfen.' Only respond with either positive, neutral or negative."}

#Creation of a new data set
student_data_oneshot = student_data_unpr
student_data_oneshot = student_data_unpr.drop("Studium_Label", axis=1)

results=[]
results_df = []

# Sending labeling requests to GPT
for index, eintrag in student_data_oneshot.iterrows():
    for inhalt in eintrag:
        response = client.chat.completions.create(
        model="gpt-3.5-turbo",
            messages=[
                initial_message,
                {"role": "user", "content": inhalt}
            ],
            #Setting temperature to ensure objective responses
            temperature=0,
            #Setting the seed parameter to ensure more consistent results
            seed = 123
        )
        
        # Saving of responses 
        results.append({"index": index, "oneshotSentimentPrediction": response.choices[0].message.content.lower()})

# Conversion of responses into dataframe
results_df = pd.DataFrame(results)



# [3.1] One shot evaluation 
In the second evaluation, GPT persists in demonstrating a bias towards positive categorization, with a reduced precision of 0.78 for positive sentiments, compared to the first evaluation. However, despite this bias, the overall F1 score shows an improvement, increasing by 4%. Nevertheless, GPT continues to struggle with accurately categorizing negative sentiments, correctly identifying only 27 out of 132 negatively labeled datasets.



In [ ]:
#Merging target labels with predicted labels and dropping of duplicated row
results_df = results_df.drop("index", axis=1)
student_data_oneshot = pd.concat([student_data_unpr, results_df], axis=1)
student_data_oneshot = student_data_oneshot[student_data_oneshot["Studium_Label"].notnull()]

#Adding of altered predictions to a dataframe in order to save all predictions in an excel
oneshot_prediction = student_data_oneshot["oneshotSentimentPrediction"]
student_data = pd.concat([student_data, altered_oneshot], axis=1)


#Convertion of all values into strings
for x in student_data_oneshot:
    student_data[x] = student_data[x].astype(str)



In [ ]:
#saving of data into an excel sheet
student_data.to_excel("Evaluation.xlsx", index=False) 

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, classification_report
import matplotlib.pyplot as plt

#Creation of confusion matrix
confusion_mat = confusion_matrix(student_data_oneshot['Studium_Label'], student_data_oneshot['oneshotSentimentPrediction'], labels=['positive', 'negative', 'neutral'])
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_mat, display_labels=['positive', 'negative', 'neutral'])
disp.plot()
plt.show()

#Accuracy
print(f1_score(student_data_oneshot['Studium_Label'], student_data_oneshot['oneshotSentimentPrediction'], labels=['positive', 'negative', 'neutral'], average='micro'))

#Creation of presicion, recall, F1 and support
print(classification_report(
    student_data_oneshot['Studium_Label'],
    student_data_oneshot['oneshotSentimentPrediction'],
    output_dict=False,
    target_names=['positive', 'negative', 'neutral']
))